# Summary

xxx

----

# Imports

In [29]:
%run imports.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-08-25 21:40:54.671985


In [30]:
%run db.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


MySQL database already running...


2016-08-25 21:40:54.843022


In [31]:
NOTEBOOK_NAME = 'clinvar'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Export from remote db

In [80]:
DB_URL = kmtools.db_tools.parse_connection_string(db_remote.connection_string)['db_url']

In [81]:
# SQL query
sql_query = r"""\
SELECT
    us.uniprot_acc,
    mutation,
    ddg_exp,
    del_score_exp,
    del_class_exp
INTO OUTFILE '/tmp/{}.tsv' 
    FIELDS TERMINATED BY '\t' OPTIONALLY ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
FROM (
    SELECT
        v.mutationassessor_uniprotid uniprot_id,
        v.mutationassessor_variant mutation,
        null ddg_exp,
        v.mutationassessor_score del_score_exp,
        0 del_class_exp
    FROM clinvar_local.clinvar_benign c
    JOIN dbnsfp.variant v ON (c.id = v.rs_dbsnp146)
    WHERE v.mutationassessor_uniprotid IS NOT NULL AND v.mutationassessor_variant IS NOT NULL
        UNION ALL
    SELECT 
        v.mutationassessor_uniprotid uniprot_id,
        v.mutationassessor_variant mutation,
        null ddg_exp,
        v.mutationassessor_score del_score_exp,
        1 del_class_exp
    FROM clinvar_local.clinvar c
    JOIN dbnsfp.variant v ON (c.id = v.rs_dbsnp146)
    WHERE v.mutationassessor_uniprotid IS NOT NULL AND v.mutationassessor_variant IS NOT NULL
) t1
JOIN uniprot_kb_proteomes.UP000005640_9606_fasta us USING (uniprot_id);
""".format(NOTEBOOK_NAME)
print_sql(sql_query)

In [82]:
result = db_remote.engine.execute(sql_query)

# Load to local db

In [83]:
!rsync -av {DB_URL}:/tmp/{NOTEBOOK_NAME}.tsv {NOTEBOOK_NAME}/{NOTEBOOK_NAME}.tsv

receiving incremental file list
clinvar.tsv

sent 23,233 bytes  received 5,435,550 bytes  727,837.73 bytes/sec
total size is 5,434,125  speedup is 1.00


In [84]:
!git lfs track {NOTEBOOK_NAME}/{NOTEBOOK_NAME}.tsv

clinvar/clinvar.tsv already supported


In [85]:
!git add {NOTEBOOK_NAME}/{NOTEBOOK_NAME}.tsv -f

There are some dulplicate rows for some reason. 

It is fastest to load the data into a temporary table and group inside the database.

In [170]:
t = db.import_file(
    op.join(NOTEBOOK_NAME, NOTEBOOK_NAME + '.tsv'),
    tablename=NOTEBOOK_NAME + '_tmp',
    names=['uniprot_id', 'uniprot_mutation', 'ddg_exp', 'del_score_exp', 'del_class_exp'],
    sep='\t',
    na_values=['\\N'],
    keep_tmp=True,
)

In [171]:
t.name

'clinvar_tmp'

In [172]:
db.engine.execute("DROP TABLE IF EXISTS {}".format(NOTEBOOK_NAME))

In [173]:
db.engine.execute("""\
CREATE TABLE {} AS (
    SELECT 
    uniprot_id, 
    uniprot_mutation, 
    NULL ddg_exp, 
    AVG(del_score_exp) del_score_exp, 
    ROUND(AVG(del_class_exp)) del_class_exp
    FROM {}
    GROUP BY uniprot_id, uniprot_mutation
)
""".format(NOTEBOOK_NAME, NOTEBOOK_NAME + '_tmp'))

In [174]:
db.engine.execute("DROP TABLE IF EXISTS {}".format(NOTEBOOK_NAME + '_tmp'))

In [175]:
t.name = t.name.replace('_tmp', '')

In [176]:
t.add_idx_column()

192948

In [177]:
t.create_indexes([
        (['uniprot_id', 'uniprot_mutation'], True),
    ])

In [178]:
t.compress()

File size before: 7.25 MB
File size after: 2.35 MB
File size savings: 2.35 MB (32.44 %)


In [186]:
!echo $NOTEBOOK_NAME

clinvar


# Git LFS

In [190]:
!git lfs track ./mysqld/{os.environ['DB_SCHEMA']}/{NOTEBOOK_NAME}.*

Tracking ./mysqld/mutation_sets/clinvar.MYD
Tracking ./mysqld/mutation_sets/clinvar.MYI
Tracking ./mysqld/mutation_sets/clinvar.frm


In [192]:
!git add -f ./mysqld/{os.environ['DB_SCHEMA']}/{NOTEBOOK_NAME}.*